In [17]:
# Running the Packages and parameters script which imports all the necessary packages and speifies parmeters for plotting
# Also specifying the order of the treatments when plotting so they all get the same color in every graph
%run packages&parameters.ipynb
processors=1

In [18]:
plt.rcParams['image.cmap'] = 'jet'
os.getcwd()

'C:\\Users\\piamaril\\Python\\Scripts\\200823_Test'

In [19]:
path = Path('C:\\Users\\piamaril\\Python\\Scripts\\200823_Test')# PIA Manually change the path to the folder including the outputdata
#For some reason the path name requires double backslashes here.

dirs = [e for e in path.iterdir() if e.is_dir()]
for c,e in enumerate (dirs):
    print(c,e)

0 C:\Users\piamaril\Python\Scripts\200823_Test\.ipynb_checkpoints
1 C:\Users\piamaril\Python\Scripts\200823_Test\20200613-214101
2 C:\Users\piamaril\Python\Scripts\200823_Test\20200823-112806
3 C:\Users\piamaril\Python\Scripts\200823_Test\mods
4 C:\Users\piamaril\Python\Scripts\200823_Test\__pycache__


In [20]:
str(Path.cwd())

'C:\\Users\\piamaril\\Python\\Scripts\\200823_Test'

In [21]:
os.getcwd()

'C:\\Users\\piamaril\\Python\\Scripts\\200823_Test'

In [22]:
# PIA specify which of the dirs listed above that the script should use (the one containing the pickle files from script 1)

data_path = dirs[2]

# C:\Users\piamaril\Python\Scripts\200823_Test\20200823-112806
# C:\Users\piamaril\Python\Scripts\200612\20200613-214101
#'D:/Forskning/Tracking/Intravital/Demo Injury for Python/200116'
#'C:/Users/Human/Google Drive/IVM sharing/IVM Tracking/Data/Demo experiments 200114'
#'D:/Forskning/Tracking/Intravital/Demo Injury for Python'
#C:/Users/Lars2/Desktop/IVM sharing/IVM Tracking/Data'

In [25]:
files=fl.get_objectDF_files(dirs[2]) # PIA change to correct dirs[x] here too
#'C:\\Users\\Human\\Google Drive\\IVM sharing\\IVM Tracking\\Scripts\\New fl 200116')#dirs[3])
print(f"There are {len(files)} pointcloud files in the data_path \nList of files in data_path:")
for c, value in enumerate(files,0):
    print(c, value)

There are 1 pointcloud files in the data_path 
List of files in data_path:
0 C:\Users\piamaril\Python\Scripts\200823_Test\20200823-112806\200519_IVMTR69_Inj4_dmso_exp3.20200823-114141.df.pkl


In [26]:
# PIA change the range depending on how many injuries in data set. If a test injury use flist=[0] 
#If a list of injuries of example 66 injuries: write (1,65,1) - IS THIS CORRECT??? (0,65,1)???
#Put hashtag on the one that you are not using
flist=[0]
#flist=list(range(1,65,1))

In [27]:
ls_df=[]
for n in flist:
    pc_=pd.read_pickle(files[n])
    print(f"Observations in file number {n} = {pc_.shape[0]}")
    ls_df.append(pc_)
pc=pd.concat(ls_df)
print(f"Total number of observations = {pc.shape[0]}")
print(f"Average number of observations per exp = {int(pc.shape[0]/len(flist))}")

Observations in file number 0 = 215958
Total number of observations = 215958
Average number of observations per exp = 215958


In [28]:
pc.head(5).T

,0,1,2,3,4
pid,0,1,2,3,4
path,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2
frame,0,0,0,0,0
time,0,0,0,0,0
x,23.625,26.6118,27.7455,32.4176,40.7324
y,149.75,470.882,252.745,305.593,290.549
z,18,7,9,8,6
xs,-129.101,-13.5337,-90.9078,-70.3985,-72.7979
ys,-13.012,100.425,22.2426,39.4201,31.0913
zs,36,14,18,16,12


In [29]:
print('Columns ', pc.columns, '\nDimensions ',pc.shape, '\nFrames ', len(pd.unique(pc.frame)))

Columns  Index(['pid', 'path', 'frame', 'time', 'x', 'y', 'z', 'xs', 'ys', 'zs', 'size',
       'int_mean', 'int_max', 'c0_mean', 'c0_max', 'c1_mean', 'c1_max', 'zf'],
      dtype='object') 
Dimensions  (215958, 18) 
Frames  194


# Load PC collection

In [30]:
print(pd.unique(pc.path))

['D:\\Test injury\\200519_IVMTR69_Inj4_dmso_exp3.nd2']


# Ytterligare Filter 

In [31]:
#help(np.geomspace)

In [32]:
# OM DET BEHÖVS, renso objekt från mindre objekt (skräp), och se hur många objekt före och efter


print(f"{pc.shape[0]} \t\t objects\n")
for n in list(np.around(np.geomspace(4, 1024, num=9)).astype(int)):
        if n<129:
            print(f"{len(pc[pc['size']<n])} \t\t smaller than \t\t {n} \t\t pixels")
        else:
            print(f"{len(pc[pc['size']>n])} \t\t larger than \t\t {n} \t\t pixels")
#print(f"{len(pc[pc['size']>1024])} \t\t larger than \t\t 1024 \t\t pixels")
    #print(f"There are {k}")
    #print('hello')  

    
    
    

215958 		 objects

2841 		 smaller than 		 4 		 pixels
6951 		 smaller than 		 8 		 pixels
14768 		 smaller than 		 16 		 pixels
31136 		 smaller than 		 32 		 pixels
80592 		 smaller than 		 64 		 pixels
149286 		 smaller than 		 128 		 pixels
13928 		 larger than 		 256 		 pixels
836 		 larger than 		 512 		 pixels
1 		 larger than 		 1024 		 pixels


# Reset index, drop columns

In [33]:
pc['pid']=list(range(pc.shape[0]))

In [34]:
pc.head(3).T

,0,1,2
pid,0,1,2
path,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2
frame,0,0,0
time,0,0,0
x,23.625,26.6118,27.7455
y,149.75,470.882,252.745
z,18,7,9
xs,-129.101,-13.5337,-90.9078
ys,-13.012,100.425,22.2426
zs,36,14,18


In [35]:
pc.tail(3).T


,215955,215956,215957
pid,215955,215956,215957
path,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2
frame,193,193,193
time,596.907,596.907,596.907
x,494.741,495.446,499.667
y,75.7069,417.054,221.111
z,3,18,16
xs,12.4787,134.44,66.0797
ys,-207.393,-85.9343,-157.304
zs,6,36,32


In [36]:
pc.set_index('pid')

,path,frame,time,x,y,z,xs,ys,zs,size,int_mean,int_max,c0_mean,c0_max,c1_mean,c1_max,zf
pid,,,,,,,,,,,,,,,,,
0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.000000,23.625000,149.750000,18.0,-129.100711,-13.011968,36.0,8.0,22.500000,32,167.750000,248.0,161.500000,212.0,30.0
1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.000000,26.611765,470.882353,7.0,-13.533677,100.425161,14.0,85.0,60.317647,118,232.282353,701.0,172.917647,295.0,8.0
2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.000000,27.745455,252.745455,9.0,-90.907753,22.242634,18.0,55.0,33.345455,61,170.836364,386.0,169.581818,252.0,12.0
3,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.000000,32.417582,305.593407,8.0,-70.398549,39.420075,16.0,91.0,57.164835,101,189.593407,358.0,175.582418,296.0,10.0
4,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.000000,40.732394,290.549296,6.0,-72.797929,31.091281,12.0,71.0,83.718310,156,254.126761,811.0,208.211268,343.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215953,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,596.907216,484.625000,386.375000,21.0,119.642568,-93.014510,42.0,16.0,22.812500,35,120.187500,158.0,180.312500,217.0,36.0
215954,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,596.907216,487.541176,378.035294,25.0,117.708769,-97.027877,50.0,85.0,23.011765,42,134.211765,223.0,153.941176,239.0,44.0
215955,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,596.907216,494.741379,75.706897,3.0,12.478660,-207.392555,6.0,58.0,45.413793,80,231.879310,584.0,177.706897,302.0,0.0


In [37]:
pc[['size','c0_max','c1_max']].describe()

,size,c0_max,c1_max
count,215958.000000,215958.000000,215958.000000
mean,107.905171,552.739959,300.487303
std,87.740790,445.993511,189.650792
min,1.000000,23.000000,49.000000
25%,46.000000,292.000000,240.000000
50%,88.000000,408.000000,262.000000
75%,144.000000,647.000000,292.000000
max,1099.000000,4095.000000,2978.000000


In [38]:
pc.tail(3).T

,215955,215956,215957
pid,215955,215956,215957
path,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2
frame,193,193,193
time,596.907,596.907,596.907
x,494.741,495.446,499.667
y,75.7069,417.054,221.111
z,3,18,16
xs,12.4787,134.44,66.0797
ys,-207.393,-85.9343,-157.304
zs,6,36,32


In [39]:
pc.head()

,pid,path,frame,time,x,y,z,xs,ys,zs,size,int_mean,int_max,c0_mean,c0_max,c1_mean,c1_max,zf
0,0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,23.625000,149.750000,18.0,-129.100711,-13.011968,36.0,8.0,22.500000,32,167.750000,248.0,161.500000,212.0,30.0
1,1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,26.611765,470.882353,7.0,-13.533677,100.425161,14.0,85.0,60.317647,118,232.282353,701.0,172.917647,295.0,8.0
2,2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,27.745455,252.745455,9.0,-90.907753,22.242634,18.0,55.0,33.345455,61,170.836364,386.0,169.581818,252.0,12.0
3,3,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,32.417582,305.593407,8.0,-70.398549,39.420075,16.0,91.0,57.164835,101,189.593407,358.0,175.582418,296.0,10.0
4,4,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,40.732394,290.549296,6.0,-72.797929,31.091281,12.0,71.0,83.718310,156,254.126761,811.0,208.211268,343.0,6.0


In [40]:
pc.shape

(215958, 18)


# Tracking

In [41]:
import trackpy as tp

In [42]:
# Function for tracking. Seach range = 3 um with these settings - change?
def track(pct):
    search_range = 3 #4
    #pred = tp.predict.NearestVelocityPredict()#Predikterar position utifran tidigare hastighet
    #linked_pc = pred.link_df(pct, search_range, pos_columns=['xs', 'ys', 'zs'], memory=1)
    linked_pc = tp.link_df(pct, search_range, pos_columns=['xs', 'ys', 'zs'], memory=1)
    #pred = trackpy.predict.NearestVelocityPredict()
    #tr = pred.link_df(pandas.concat(frames), 0.5)
    return(linked_pc)

In [43]:
# Applying tracking function to find trajectories ?
t_grp=pc.reset_index().groupby(['path']).apply(track)


Frame 193: 1806 trajectories present.


In [44]:
tracks=t_grp.drop(['path'], axis=1).reset_index().sort_values('pid')

In [45]:
dv_=[]

p_grp=tracks.groupby(['path', 'particle'])
for i, gr in p_grp:
    grp=gr.sort_values('frame')
    dvx = np.append(np.diff(grp['xs']), np.nan)
    dvy = np.append(np.diff(grp['ys']), np.nan)
    dvz = np.append(np.diff(grp['zs']), np.nan)
    df=grp.pid.to_frame()
    df['dvx']=dvx
    df['dvy']=dvy
    df['dvz']=dvz
    df['particle']=grp.particle
    
    dv_.append(df)

In [46]:
dv=pd.concat(dv_, axis=0)

In [47]:
dv=dv.sort_values('pid')

In [48]:
dv['dv']=(dv.dvx**2+dv.dvy**2+dv.dvz**2)**0.5

In [49]:
tracks_grp=dv.sort_values('pid').reset_index()
tracks_grp=tracks_grp[['pid', 'dvx', 'dvy', 'dvz', 'dv', 'particle']].set_index('pid')

In [50]:
tracks_grp.tail(20)

,dvx,dvy,dvz,dv,particle
pid,,,,,
215938,NaN,NaN,NaN,NaN,52318
215939,NaN,NaN,NaN,NaN,52421
215940,NaN,NaN,NaN,NaN,52494
215941,NaN,NaN,NaN,NaN,52335
215942,NaN,NaN,NaN,NaN,52422
215943,NaN,NaN,NaN,NaN,52466
215944,NaN,NaN,NaN,NaN,52493
215945,NaN,NaN,NaN,NaN,52409
215946,NaN,NaN,NaN,NaN,52507


# Nearest Neighbours

In [51]:
def nb(pc):
    nb_count=3
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>nb_count:
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):

            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nb_d_' + str(i)]=nb_dist
            key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_idx['nb_i_0']=a
    
    df=pd.DataFrame(key_idx)
    df=pd.concat([p1i, df], axis=1)
    
    return df


In [52]:
t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nb).reset_index()

In [53]:
t_grp=t_grp.set_index('pid')

In [54]:
t_grp.head(3)#.T

,path,frame,level_2,nb_i_0,nb_i_1,nb_i_2,nb_d_0,nb_d_1,nb_d_2
pid,,,,,,,,,
0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0,11,6,10,23.893015,24.893538,46.898780
1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,1,8,17,5,16.642097,37.753830,47.146773
2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,2,7,4,10,10.892403,21.030080,24.974868


In [55]:
nb_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

In [56]:
nb_grp.head(10)

,nb_i_0,nb_i_1,nb_i_2,nb_d_0,nb_d_1,nb_d_2
pid,,,,,,
0,11,6,10,23.893015,24.893538,46.898780
1,8,17,5,16.642097,37.753830,47.146773
2,7,4,10,10.892403,21.030080,24.974868
3,4,2,13,9.545986,26.827075,28.184654
4,3,2,7,9.545986,21.030080,22.297389
5,9,12,16,11.071850,22.130042,26.933566
6,10,0,14,22.430609,24.893538,26.711310
7,2,10,4,10.892403,15.938580,22.297389
8,1,17,15,16.642097,25.394929,39.989774


# Nearest Neighbours - Average

In [57]:
def nb(pc):
    nb_count=3
    nba_list=[5,10,15]
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>np.array(nba_list).max():
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        #dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in nba_list:

            nb_dist=(dmap_sorted[1:(i+1),:]).mean(axis=0)
            #nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nba_d_' + str(i)]=nb_dist
            #key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        #key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_dist[('nba_d_' + str(nba_list[0]))]=a
    
    df=pd.DataFrame(key_dist)
    df=pd.concat([p1i, df], axis=1)
    
    return df


In [58]:
t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nb).reset_index()

In [59]:
t_grp=t_grp.set_index('pid')

In [60]:
nba_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

In [61]:
nba_grp.head(10)

,nba_d_5,nba_d_10,nba_d_15
pid,,,
0,38.551085,46.197638,49.775324
1,40.098898,48.319110,52.571990
2,24.039777,35.001649,42.134761
3,26.459826,34.528054,38.839893
4,22.590653,32.203828,36.733403
5,23.386661,27.694849,31.029261
6,26.991990,33.194832,38.616431
7,22.426022,30.263917,36.618901
8,34.253868,41.276820,44.661945


# Nearest Neighbours -  all colors

def nbc(pc):
    nb_count=3
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>nb_count:
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):

            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc_d_' + str(i)]=nb_dist
            key_idx['nbc_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_idx['nbc_i_0']=a
        
    
    df=pd.DataFrame(key_idx)
    df=pd.concat([p1i, df], axis=1)
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbc).reset_index()

t_grp=t_grp.set_index('pid')

nbc_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

nbc_grp.sort_index().head()

# Nearest Neighbours -  all colors - Average

def nbca(pc):
    nb_count=3
    nba_list=[5,10,15]
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>np.array(nba_list).max():
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        #dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in nba_list:

            nb_dist=(dmap_sorted[1:(i+1),:]).mean(axis=0)
            #nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbca_d_' + str(i)]=nb_dist
            #key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        #key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_dist[('nbca_d_' + str(nba_list[0]))]=a
    
    df=pd.DataFrame(key_dist)
    df=pd.concat([p1i, df], axis=1)
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbca).reset_index()

t_grp=t_grp.set_index('pid')

nbca_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

nbca_grp.sort_index().head(10)

# Nearest Neighbours -  second color only

p=0
pc1=pc[pc.path==(pd.unique(pc.path)[p])]
#pc1=df[df.c==625]
pc1=pc1[pc1.frame==20]

pd.unique(pc.c)

def nbc2(pc1):
    c1=470
    c2=555

    nb_count=3
    key_dist={}
    key_idx={}


    #print(len(pc))
    if (len(pc1[pc1.c==c1])>nb_count) & (len(pc1[pc1.c==c2])>nb_count):
        pos1=pc1[pc1.c==c1]
        #print(len(pos1))
        p1i=pos1.reset_index().pid
        pos1=pos1[['xs','ys','zs']].values
        pos2=pc1[pc1.c==c2]
        #print(len(pos2))
        p2i=pos2.reset_index().pid
        pos2=pos2[['xs','ys','zs']].values

        dmap=spatial.distance.cdist(pos1, pos2)

        key_dist={}
        key_idx={}
        dmap_sorted=np.sort(dmap.T, axis=0)
        dmap_idx_sorted=np.argsort(dmap.T, axis=0)
        for i in range(nb_count):
            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc2_d_' + str(i)]=nb_dist
            key_idx['nbc2_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
        p1=pd.DataFrame(key_idx)
        p1=pd.concat([p1,p1i], axis=1).set_index('pid')

        key_dist={}
        key_idx={}
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):
            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc2_d_' + str(i)]=nb_dist
            key_idx['nbc2_i_' + str(i)]=pd.Series(nb_idx).map(p2i).values.astype('int').tolist()
        key_idx.update(key_dist)
        p2=pd.DataFrame(key_idx)
        p2=pd.concat([p2,p2i], axis=1).set_index('pid')
        
        df=pd.concat([p1,p2], axis=0).sort_index()

    else:
        a = np.empty((len(pc1)))
        a[:] = np.nan
        key_idx['nbc2_i_0']=a
        pi=pc1.reset_index().pid
        df=pd.DataFrame(key_idx)
        df=pd.concat([df,pi], axis=1).set_index('pid')
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbc2)

t_grp=t_grp.reset_index().set_index('pid')

nbc2_grp=t_grp.drop(['path', 'frame'], axis=1)

nbc2_grp.head()

# DBSCAN - is this a measure of object clustering? We have not used this so far but we might find interesting inforamation here especially in the arms. Optimise this code furhter?

In [62]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.preprocessing import StandardScaler

def cluster(pc):
    
    
    min_samples=5
    
    eps_list=[5,7.5,10,15,20]
    
    cl_=[]
    
    #X=pos[:,:-1] # 2D
    X=pc[['xs','ys','zs']].values # 3D
    
    for eps in eps_list:
        # Compute DBSCAN
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        #print('Estimated number of clusters: %d' % n_clusters_)
        
        cl_.append(pd.DataFrame({('cl_idx_' + str(eps) ) : (labels)}))
                     
    

        
    ##############################################################################
    # Plot result

    # Black removed and is used for noise instead.
    
#    unique_labels = set(labels)
#    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
#    for k, col in zip(unique_labels, colors):
#        if k == -1:
#            # Black used for noise.
#            col = 'k'

#        class_member_mask = (labels == k)

#        xy = X[class_member_mask & core_samples_mask]
#       plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=8)

#        xy = X[class_member_mask & ~core_samples_mask]
#        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=3)

#    plt.title('Estimated number of clusters: %d' % n_clusters_)
#    plt.show()
    cl_.append(pc.reset_index().pid)
    return (pd.concat(cl_, axis=1))

In [63]:
t_grp=pc.groupby(['path', 'frame']).apply(cluster).reset_index()

In [64]:
cl_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)


In [65]:
cl_grp=cl_grp.set_index('pid')

In [66]:
cl_grp.head(10)

,cl_idx_5,cl_idx_7.5,cl_idx_10,cl_idx_15,cl_idx_20
pid,,,,,
0,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,0
6,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1


# DBSCAN 2

In [67]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.preprocessing import StandardScaler

def cluster_d(pc):
    
    min_samples=5
    
    eps_list=np.arange(3, 31, 2)
    
    cl_=[]
    
    #X=pos[:,:-1] # 2D
    X=pc[['xs','ys','zs']].values # 3D
    
    for eps in eps_list:
        # Compute DBSCAN
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        #print('Estimated number of clusters: %d' % n_clusters_)
        
        labels[labels>-1]=eps
        labels[labels==-1]=42
        
               
        cl_.append(pd.DataFrame({('cl_idx_' + str(eps) ) : (labels)}))
    cld_grp=pd.concat(cl_, axis=1)
    cld_grp=pd.DataFrame({('cld' ) : (cld_grp.min(axis=1))})
    
                     
        

        
    ##############################################################################
    # Plot result

    # Black removed and is used for noise instead.
    
#    unique_labels = set(labels)
#    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
#    for k, col in zip(unique_labels, colors):
#        if k == -1:
#            # Black used for noise.
#            col = 'k'

#        class_member_mask = (labels == k)

#        xy = X[class_member_mask & core_samples_mask]
#       plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=8)

#        xy = X[class_member_mask & ~core_samples_mask]
#        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=3)

#    plt.title('Estimated number of clusters: %d' % n_clusters_)
#    plt.show()
    cld_grp['pid']=(pc.reset_index().pid)
    return (cld_grp)

In [68]:
t_grp=pc.groupby(['path', 'frame']).apply(cluster_d).reset_index()

In [69]:
t_grp.tail()

,path,frame,level_2,cld,pid
215953,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,1801,11,215953
215954,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,1802,11,215954
215955,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,1803,21,215955
215956,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,1804,13,215956
215957,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,193,1805,42,215957


In [70]:
cld_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)


In [71]:
cld_grp=cld_grp.set_index('pid')

In [72]:
cld_grp.head(10)

,cld
pid,
0,29
1,42
2,25
3,27
4,27
5,21
6,25
7,25
8,42


# Stability 2, pc only

In [73]:
def do_tstab(tgrp):
    ocp_=[]
    first=True
    #print(len(tgrp))
    for i, grp in tgrp.groupby(['frame']):
        pos=grp[['xs','ys','zs']].values
        if first:
            first=False
        else:
            dmap=spatial.distance.cdist(t1, pos)
            dmap_sorted=np.sort(dmap, axis=1)
            data=dmap_sorted[:,0]

            ocp_.append(pd.DataFrame({'stab' : data}))#, 'pid': grp.pid}))

        t1=pos
    data=np.zeros(len(pos))
    data[:]=np.nan
    ocp_.append(pd.DataFrame({'stab' : data}))#, 'pid':grp.pid}))

    ocp=pd.concat(ocp_, axis=0).reset_index()
    ocp['pid']=tgrp.reset_index().pid
    
    return ocp

In [74]:
t_grp=pc.groupby(['path']).apply(do_tstab)

In [75]:
stab_grp=t_grp.reset_index()[['stab', 'pid']]

In [76]:
stab_grp=stab_grp.set_index('pid').sort_index()

In [77]:
stab_grp.head(10)

,stab
pid,
0,18.467347
1,28.211953
2,23.504878
3,13.857621
4,17.438415
5,10.375184
6,8.691322
7,31.272748
8,17.933248


# Point depth

def depth(pos):
    
    fill_dist=15
    
    pos_max=pos[['xs', 'ys', 'zf']].max()

    zsize=int(pos_max['zf']*1.5)
    xsize=int(pos_max['xs']*1.1)
    ysize=int(pos_max['ys']*1.1)

    pcc=np.zeros((xsize+2,ysize+2,zsize+2))
    pc_pos=pos[['xs', 'ys', 'zf']].values

    #zfloor=int(np.percentile(pc_pos[:,2],2))
    zfloor=1
    pc_pos[pc_pos<0]=0
    pc_pos=pc_pos.astype('int').T
    
    pc_pos=pc_pos.tolist()
      
    pcc[pc_pos]=1
    pcc[:,:,:zfloor]=1

    pcd=ndimage.morphology.distance_transform_edt(pcc==0)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pcd[300,:,:]).T, cmap='jet', vmin=0, vmax=20)
    #plt.colorbar()

    pca=pcd<fill_dist

    pca=ndimage.binary_fill_holes(pca)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pca[300,:,:]).T, cmap='jet', vmin=0, vmax=2)
    #plt.colorbar()

    pcad=ndimage.morphology.distance_transform_edt(pca)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pcad[300,:,:]).T, cmap='jet', vmin=0, vmax=40)
    #plt.colorbar()

    pdepth=pcad[pc_pos]-fill_dist
           
    depth_grp=pd.DataFrame({'depth' : (pdepth)})
    
    depth_grp['pid']=pos.reset_index().pid
    
    return depth_grp


def depth(pos_grp):
    
    fill_dist=15
    
    pos_max=pos_grp[['xs', 'ys', 'zf']].max()

    zsize=int(pos_max['zf']*1.5)
    xsize=int(pos_max['xs']*1.1)
    ysize=int(pos_max['ys']*1.1)

    depth_grp_=[]
    
    for i, pos in pos_grp.groupby('frame'):
    
        pcc=np.zeros((xsize+2,ysize+2,zsize+2))
        
        pc_pos=pos[['xs', 'ys', 'zf']].values
        
        zfloor=1
        pc_pos[pc_pos<0]=0
        pc_pos=pc_pos.astype('int').T

        pc_pos=pc_pos.tolist()

        pcc[pc_pos]=1
        pcc[:,:,:zfloor]=1

        pcd=ndimage.morphology.distance_transform_edt(pcc==0)
        pca=pcd<fill_dist

        pca=ndimage.binary_fill_holes(pca)
        pcad=ndimage.morphology.distance_transform_edt(pca)
        pdepth=pcad[pc_pos]-fill_dist

        depth_grp=pd.DataFrame({'depth' : (pdepth)})

        depth_grp['pid']=pos.reset_index().pid
        
        depth_grp_.append(depth_grp)
    
    depth_grp_=pd.concat(depth_grp_, axis=0)
    return depth_grp_


t_grp=pc.groupby(['path']).apply(depth).reset_index()   

depth_grp=t_grp[['pid', 'depth']]

depth_grp=depth_grp.set_index('pid').sort_index()

depth_grp.tail(10)

# Export

In [80]:
# Pia - change base name to current folder. 
# Pia Make sure there is no subfolder called mods if the folder was copied from other experiment??? We are making a mods subfolder
base_name = '200823_Test'

In [81]:
folder='./mods/'
#/tmp/year
os.mkdir(folder)
base_name=folder + base_name 

In [82]:
# Saving modified point-cloud data as pickle file: file name pc_mod
#pc=pc.set_index('pid')
pc.to_pickle(base_name + '_pc_mod')
print(len(pc))
pc.head()

215958


,pid,path,frame,time,x,y,z,xs,ys,zs,size,int_mean,int_max,c0_mean,c0_max,c1_mean,c1_max,zf
0,0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,23.625000,149.750000,18.0,-129.100711,-13.011968,36.0,8.0,22.500000,32,167.750000,248.0,161.500000,212.0,30.0
1,1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,26.611765,470.882353,7.0,-13.533677,100.425161,14.0,85.0,60.317647,118,232.282353,701.0,172.917647,295.0,8.0
2,2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,27.745455,252.745455,9.0,-90.907753,22.242634,18.0,55.0,33.345455,61,170.836364,386.0,169.581818,252.0,12.0
3,3,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,32.417582,305.593407,8.0,-70.398549,39.420075,16.0,91.0,57.164835,101,189.593407,358.0,175.582418,296.0,10.0
4,4,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,40.732394,290.549296,6.0,-72.797929,31.091281,12.0,71.0,83.718310,156,254.126761,811.0,208.211268,343.0,6.0


In [83]:
# Saving stabiliy data as pickle file:  file name _stab
stab_grp.to_pickle(base_name + '_stab')
print(len(stab_grp))
stab_grp.head()

215958


,stab
pid,
0,18.467347
1,28.211953
2,23.504878
3,13.857621
4,17.438415


In [84]:
cl_grp.sort_index().to_pickle(base_name + '_cl')
print(len(cl_grp))
cl_grp.sort_index().tail()

215958


,cl_idx_5,cl_idx_7.5,cl_idx_10,cl_idx_15,cl_idx_20
pid,,,,,
215953,-1,-1,-1,5,2
215954,-1,-1,-1,5,2
215955,-1,-1,-1,-1,-1
215956,-1,-1,-1,5,2
215957,-1,-1,-1,-1,-1


In [85]:
cld_grp.sort_index().to_pickle(base_name + '_cld')
print(len(cld_grp))
cld_grp.sort_index().tail()

215958


,cld
pid,
215953,11
215954,11
215955,21
215956,13
215957,42


In [86]:
# Saving tracks as pickle file: file name_tracks
#tracks_grp=tracks_grp.set_index('pid')
tracks_grp.to_pickle(base_name + '_tracks')
print(len(tracks_grp))
tracks_grp.head()

215958


,dvx,dvy,dvz,dv,particle
pid,,,,,
0,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,191
2,NaN,NaN,NaN,NaN,190
3,NaN,NaN,NaN,NaN,189
4,NaN,NaN,NaN,NaN,188


depth_grp.to_pickle(base_name + '_depth')
print(len(depth_grp))
depth_grp.head()

## Neighbours

In [87]:
# Saving nearest neighbours data as pickle: file name_nb
nb_grp.to_pickle(base_name + '_nb')
print(len(nb_grp))
nb_grp.tail()

215958


,nb_i_0,nb_i_1,nb_i_2,nb_d_0,nb_d_1,nb_d_2
pid,,,,,,
215953,215954,215949,215951,9.156784,9.741285,10.971964
215954,215953,215949,215944,9.156784,9.851047,16.849243
215955,215947,215952,215941,12.537575,20.892724,24.451501
215956,215951,215946,215953,9.927906,16.043296,17.466664
215957,215932,215942,215908,30.391815,30.749485,34.381764


nbc_grp.to_pickle(base_name + '_nbc')
print(len(nbc_grp))
nbc_grp.tail()

In [88]:
# Saving nearest neighbours average data as pickle: file name_nba
nba_grp.to_pickle(base_name + '_nba')
print(len(nba_grp))
nba_grp.tail()

215958


,nba_d_5,nba_d_10,nba_d_15
pid,,,
215953,12.437946,16.432626,20.164498
215954,14.790056,19.568328,23.404544
215955,24.452741,30.345192,34.010683
215956,16.591462,18.739458,20.637251
215957,34.204345,38.082884,40.926342


nbca_grp.to_pickle(base_name + '_nbca')
print(len(nbca_grp))
nbca_grp.tail()

nbc2_grp.to_pickle(base_name + '_nbc2')
print(len(nbc2_grp))
nbc2_grp.tail()

# Addon tracking - contraction component (förstör pc)

In [89]:
#pc=pc.reset_index()
pc.head()

,pid,path,frame,time,x,y,z,xs,ys,zs,size,int_mean,int_max,c0_mean,c0_max,c1_mean,c1_max,zf
0,0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,23.625000,149.750000,18.0,-129.100711,-13.011968,36.0,8.0,22.500000,32,167.750000,248.0,161.500000,212.0,30.0
1,1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,26.611765,470.882353,7.0,-13.533677,100.425161,14.0,85.0,60.317647,118,232.282353,701.0,172.917647,295.0,8.0
2,2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,27.745455,252.745455,9.0,-90.907753,22.242634,18.0,55.0,33.345455,61,170.836364,386.0,169.581818,252.0,12.0
3,3,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,32.417582,305.593407,8.0,-70.398549,39.420075,16.0,91.0,57.164835,101,189.593407,358.0,175.582418,296.0,10.0
4,4,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,0,0.0,40.732394,290.549296,6.0,-72.797929,31.091281,12.0,71.0,83.718310,156,254.126761,811.0,208.211268,343.0,6.0


In [90]:
pc1=pd.concat([pc[['path', 'xs', 'ys', 'zf', 'pid','frame']].set_index('pid'), tracks_grp], axis=1).sort_index()

In [91]:
def contract(t2):
    #xmed, ymed =(t2[t2.frame<50]['xs'].median(), t2[t2.frame<50]['ys'].median())
    #xmed, ymed =(t2['xs'].median(), t2['ys'].median())

    #t2['xm']=t2['xs']-xmed
    #t2['ym']=t2['ys']-ymed

    #t2['cont']=((-t2['xm'])*t2['dvx'] + (-t2['ym'])*t2['dvy'] + (-t2['zf'])*t2['dvz'] )/((t2['xm'])**2 + (t2['ym'])**2 + (t2['zf'])**2)**0.5
    t2['cont']=((-t2['xs'])*t2['dvx'] + (-t2['ys'])*t2['dvy'] + (-t2['zf'])*t2['dvz'] )/((t2['xs'])**2 + (t2['ys'])**2 + (t2['zf'])**2)**0.5
    t2['cont_p']=t2.cont/t2.dv
    
    
    
    return pd.DataFrame({'cont' : (t2['cont']), 'cont_p' : (t2['cont_p']), 'pid':t2['pid']})

In [92]:
cont_grp=pc1.reset_index().groupby(['path']).apply(contract)
cont_grp=cont_grp.set_index('pid').sort_index()

In [93]:
cont_grp.to_pickle(base_name + '_cont')
print(len(cont_grp))
cont_grp.head()

215958


,cont,cont_p
pid,,
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


# Nytt paket med lite nya numeriska variabler som beskriver tracking och totala rorelser 

## These functions give us the extra readouts incluidng tracknr, nrtracks, cont_tot, disp_tot, dvz_tot

In [94]:
#Skapar dataframe med grundpaket plus tracking plus kontraktions
pc1=pd.concat([pc[['path', 'xs', 'ys', 'zf', 'pid','frame']].set_index('pid'), tracks_grp,cont_grp], axis=1).sort_index()

In [95]:
pc1.head()

,path,xs,ys,zf,frame,dvx,dvy,dvz,dv,particle,cont,cont_p
pid,,,,,,,,,,,,
0,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,-129.100711,-13.011968,30.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
1,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,-13.533677,100.425161,8.0,0,NaN,NaN,NaN,NaN,191,NaN,NaN
2,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,-90.907753,22.242634,12.0,0,NaN,NaN,NaN,NaN,190,NaN,NaN
3,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,-70.398549,39.420075,10.0,0,NaN,NaN,NaN,NaN,189,NaN,NaN
4,D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2,-72.797929,31.091281,6.0,0,NaN,NaN,NaN,NaN,188,NaN,NaN


In [96]:
#Satter index pa path och particle for att kunna konkaternera dataframe senare
pc1=pc1.reset_index().set_index(['path','particle'])

#Grupperar data pa path och partikel for att kunna analysera olika partiklar for sig
grouped=pc1.groupby(['path','particle'])
#Raknar antalet observationer for varje unik partikel
counted=grouped.count()
#Variabeln nrtracks kollar hur manga tracks en viss partikel ar trackad
pc1['nrtracks']=counted.frame

#Adderar de olika variablerna och beraknar summan av alla observationer
summed=grouped.sum()
#cont_tot summerar den totala kontraktionen for en partikel
pc1['cont_tot']=summed.cont
# displ_tot summerar den totala forflyttningen for en partikel
pc1['displ_tot']=abs(summed.dvx)+abs(summed.dvy)
# dvz_tot summerar den totala forflyttningen i z-led for en partikel
pc1['dvz_tot']=summed.dvz

In [97]:
#Satter om index till pid for att ha ett unikt index for varje observation
pc1=pc1.reset_index().set_index('pid')
# Tracknr raknar vilken trackning i ordningen en viss observation ar
tracknr=pc1.groupby(['path','particle'])['frame'].rank()
pc1['tracknr']=tracknr

#Kontrollerar att de nya variablerna fungerar som de ska
pc1.loc[pc1.nrtracks>5,['path','particle','frame','tracknr','nrtracks','cont_tot','displ_tot','dvz_tot']].set_index(['path','particle','frame']).sort_index().head(20)

tracknr  \
path                                             particle frame            
D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2 24       0          1.0   
                                                          1          2.0   
                                                          2          3.0   
                                                          3          4.0   
                                                          4          5.0   
                                                          5          6.0   
                                                          6          7.0   
                                                          7          8.0   
                                                          8          9.0   
                                                          9         10.0   
                                                          10        11.0   
                                                          11        12.0   
                                                          12        13.0   
                                                          13        14.0   
                                                          14        15.0   
                                                          15        16.0   
                                                          16        17.0   
                                                          17        18.0   
                                                          18        19.0   
                                                          19        20.0   

                                                                 nrtracks  \
path                                             particle frame             
D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2 24       0           194   
                                                          1           194   
                                                          2           194   
                                                          3           194   
                                                          4           194   
                                                          5           194   
                                                          6           194   
                                                          7           194   
                                                          8           194   
                                                          9           194   
                                                          10          194   
                                                          11          194   
                                                          12          194   
                                                          13          194   
                                                          14          194   
                                                          15          194   
                                                          16          194   
                                                          17          194   
                                                          18          194   
                                                          19          194   

                                                                  cont_tot  \
path                                             particle frame              
D:\Test injury\200519_IVMTR69_Inj4_dmso_exp3.nd2 24       0      21.819923   
                                                          1      21.819923   
                                                          2      21.819923   
                                                          3      21.819923   
                                                          4      21.819923   
                                                          5      21.819923   
                                                          6      21.81992

In [98]:
# Saving these readouts as pickle file: file name_xtratracking
xtra_tracking=pc1.loc[:,['tracknr','nrtracks','cont_tot','displ_tot','dvz_tot']]
xtra_tracking.to_pickle(base_name + '_xtratracking')
print(len(xtra_tracking))
xtra_tracking.head()

215958


,tracknr,nrtracks,cont_tot,displ_tot,dvz_tot
pid,,,,,
0,1.0,1,0.0,0.0,0.0
1,1.0,1,0.0,0.0,0.0
2,1.0,1,0.0,0.0,0.0
3,1.0,1,0.0,0.0,0.0
4,1.0,1,0.0,0.0,0.0


In [99]:
os.getcwd()

'C:\\Users\\piamaril\\Python\\Scripts\\200823_Test'